In [2]:
! pip install apache_beam

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 73.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 86.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.5/261.5 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31

In [ ]:
import apache_beam as beam
p1 = beam.Pipeline()
rating=(
    p1
    | beam.io.ReadFromText('movie_ratings.txt',skip_header_lines=1)
    | beam.Map(lambda record: record.split(','))
    | beam.Filter(lambda record: float(record[2])>4)
    | beam.io.WriteToText('result.txt')
)
p1.run()

In [ ]:
ls

beam-temp-result.txt-a599d1e8e6b511ef80060242ac1c000c/  result.txt-00000-of-00001
movie_ratings.txt                                       sample_data/


In [ ]:
! cat result.txt-00000-of-00001

['3', 'SpiderMan', '4.5']


In [13]:
import apache_beam as beam
p1 = beam.Pipeline()
customer=(
    p1
    | beam.io.ReadFromText('customer.txt',skip_header_lines=1)
    | beam.Map(lambda record: record.split(','))
    | beam.Filter(lambda record: record[2]=='ny' and int(record[3])>20)
    | beam.io.WriteToText('customerresult.txt',"cs",True,1,"-SS-NN")
)
p1.run()


In [9]:
ls

customerresult.txt-00000-of-00001  customerresult.txt-01-03cs  customer.txt
customerresult.txt-00-03cs         customerresult.txt-02-03cs  sample_data/


In [15]:
! cat customerresult.txt-00-01cs

['1', 'john', 'ny', '22']
['3', 'mary', 'ny', '30']


In [19]:
import apache_beam as beam

words=['Peter','piper','pickled','peck','pepper']
def findword(element):
  if element in words:
    return True

p1 = beam.Pipeline()

freq = (
    p1
    |beam.io.ReadFromText('Peter.txt')
    |beam.FlatMap(lambda record: record.split(' '))
    |beam.Filter(findword)
    |beam.Map(lambda record: (record,1))
    |beam.CombinePerKey(sum)
    |beam.io.WriteToText('peterresult.txt')
)
p1.run()

In [20]:
ls

customerresult.txt-00000-of-00001  customerresult.txt-01-03cs  peterresult.txt-00000-of-00001
customerresult.txt-00-01cs         customerresult.txt-02-03cs  Peter.txt
customerresult.txt-00-03cs         customer.txt                sample_data/


In [21]:
! cat peterresult.txt-00000-of-00001

('Peter', 4)
('piper', 4)
('peck', 4)
('pickled', 4)
('pepper', 1)


In [22]:
import apache_beam as beam

p=beam.Pipeline()
movie_name=[
    (1,'spiderman'),
    (2,'avengers'),
    (3,'titinic'),
    (4,'grren miles'),
]

movie_ratings=[
    (1,3.5),
    (2,4),
    (3,3.5),
    (1,4.5),
    (2,4.5)
]

name = p|"Create Name Pcollection" >> beam.Create(movie_name)
rating = p|"Create Rating Pcollection" >> beam.Create(movie_ratings)

joinedRsult=((name,rating)|beam.CoGroupByKey())|beam.Map(print)

p.run()

(1, (['spiderman'], [3.5, 4.5]))
(2, (['avengers'], [4, 4.5]))
(3, (['titinic'], [3.5]))
(4, (['grren miles'], []))


In [25]:
import apache_beam as beam

p = beam.Pipeline()
number={1,2,3,4,5,6,7,8}

def partition_fn(element,num_partitions):
  return 0 if element%2==0 else 1

number_pc = p| beam.Create(number)| beam.Partition(partition_fn, 2)

number_pc[1]| 'Printing first partition' >> beam.Map(print)

p.run()

1
3
5
7


In [ ]:
import apache_beam as beam
p1 = beam.Pipeline()

class MyTransform(beam.PTransform):
  def expand(self, input_col):
    a = (
        input_col
                   | 'Calculate Sum' >> beam.Map(CalculateSum)
                   | 'Apply Formatting' >> beam.Map(FormatText)
    )
    return a

def SplitRow(input_element):
  return input_element.split(',')

def FilterBasedonCountry(countryName,input_element):
  return input_element[1]==countryName

def CalculateSum(elem):
  return elem[0],(int(elem[2])+int(elem[3])+int(elem[4]))

def FormatText(elem):
  return elem[0]+' has recieved '+str(elem[1]) +' marks'

input_collection = (
    p1
    |beam.io.ReadFromText('students_marks.txt')
    |beam.Map(SplitRow)
  )

US_pipeline = (
   input_collection
   | beam.Filter(lambda record: FilterBasedonCountry('US',record))
   | "Composite Transformation for US" >> MyTransform()
   | 'Writing results to US File' >> beam.io.WriteToText('US_Result')
)

India_pipeline = (
   input_collection
   | beam.Filter(lambda record: FilterBasedonCountry('IN',record))
   | "Composite Transformation for IN" >> MyTransform()
   |'Writing results to India File' >> beam.io.WriteToText('IN_Result')
)


p1.run()

In [ ]:
! cat US_Result-00000-of-00001

In [ ]:
! cat US_Result-00000-of-00001